In [1]:
import json
import os
import re
import math
from datetime import datetime, timedelta, time
from collections import defaultdict
from decimal import Decimal

import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.io import to_html

import yfinance as yf

from IPython.display import display, HTML

from transaction_history_processor import PortfolioHistory

In [2]:
def find_closest_date(dates, target_date):
    target_date = pd.to_datetime(target_date)
    dates = pd.to_datetime(dates)
    closest_date = min(dates, key=lambda x: abs(x - target_date))
    return closest_date.strftime('%Y-%m-%d')

NOTEBOOK_DATE = None  # Set to None to use the latest date by default

# To set a specific date, uncomment and modify the following line:
NOTEBOOK_DATE = "2023-12-19"

In [3]:
# if not os.path.exists('portfolio_history.json'):

print("Generating / updating portfolio_history.json...")
portfolio_history = PortfolioHistory()
portfolio_history.process_transaction_history(
    input_file='transaction_history.json',
    save_output=True,
    output_file='portfolio_history.json'
)
print("portfolio_history.json has been generated / updated.")

with open('portfolio_history.json', 'r') as f:
    data = json.load(f)
    sectors = data['sectors']
    portfolio_data = data['portfolios']

2024-09-01 16:14:48,095 - INFO - Successfully loaded transactions from transaction_history.json


Generating / updating portfolio_history.json...


[*********************100%%**********************]  10 of 10 completed
[*********************100%%**********************]  10 of 10 completed
2024-09-01 16:14:52,941 - INFO - Successfully fetched historical data and sector information for 20 symbols.
2024-09-01 16:14:53,010 - INFO - Portfolio history and sectors saved to portfolio_history.json
2024-09-01 16:14:53,011 - INFO - Successfully processed transaction history with 129 dates.


Split detected for NVDA on 2024-06-10. Ratio: 10.0000
portfolio_history.json has been generated / updated.


In [4]:
# Adjust NOTEBOOK_DATE if it's set but not available in the data
if NOTEBOOK_DATE:
    available_dates = sorted(portfolio_data.keys())
    if NOTEBOOK_DATE not in available_dates:
        closest_date = find_closest_date(available_dates, NOTEBOOK_DATE)
        print(f"No data available for {
              NOTEBOOK_DATE}. Using closest available date: {closest_date}")
        NOTEBOOK_DATE = closest_date

In [5]:
def process_portfolio_data(portfolio_data, date=None, sort_by='Symbol', ascending=True):
    date = NOTEBOOK_DATE or date or max(portfolio_data.keys())
    if date not in portfolio_data:
        raise ValueError(f"No data available for date: {date}")

    data = portfolio_data[date]
    holdings_data = process_holdings(data['holdings'])

    df = pd.DataFrame(holdings_data)
    df = df.sort_values(by=sort_by, ascending=ascending)

    closed_positions_df = process_closed_positions(data['closed_positions'])
    partially_sold_positions_df = process_partially_sold_positions(
        data['partially_sold_positions'])

    return {
        'date': date,
        'holdings': df,
        'summary': data['summary'],
        'closed_positions': closed_positions_df,
        'partially_sold_positions': partially_sold_positions_df,
    }

def process_holdings(holdings):
    return [
        {
            'Symbol': symbol,
            'Quantity': round(float(info['quantity']), 2),
            'Avg Cost': round(float(info['average_cost']), 2),
            'Market Price': round(float(info['market_price']), 2),
            'Total Cost': round(float(info['cost_basis']), 2),
            'Market Value': round(float(info['market_value']), 2),
            'Unrealized P/L': f"${float(info['unrealized_pnl']):,.2f} ({float(info['unrealized_return'])*100:.2f}%)",
            'Daily Change': f"${float(info['daily_change']):,.2f} ({float(info['daily_return'])*100:.2f}%)"
        }
        for symbol, info in holdings.items()
    ]

def process_closed_positions(closed_positions):
    df = pd.DataFrame(closed_positions).T.reset_index()
    df.columns = ['Symbol', 'Close Date', 'Quantity',
                  'Cost Basis', 'Realized P/L', 'Realized Return']
    df['Realized P/L'] = df['Realized P/L'].apply(
        lambda x: f"${float(x):,.2f}")
    df['Realized Return'] = df['Realized Return'].apply(
        lambda x: f"{float(x)*100:.2f}%")  # Multiply by 100 here
    return df

def process_partially_sold_positions(partially_sold_positions):
    df = pd.DataFrame(partially_sold_positions).T.reset_index()
    df.columns = ['Symbol', 'Last Sell Date', 'Quantity',
                  'Cost Basis', 'Realized P/L', 'Realized Return']
    df['Realized P/L'] = df['Realized P/L'].apply(
        lambda x: f"${float(x):,.2f}")
    df['Realized Return'] = df['Realized Return'].apply(
        lambda x: f"{float(x)*100:.2f}%")  # Multiply by 100 here
    return df

def display_portfolio_analysis(processed_data):
    date = processed_data['date']
    df = processed_data['holdings']
    summary = processed_data['summary']
    closed_positions_df = processed_data['closed_positions']
    partially_sold_positions_df = processed_data['partially_sold_positions']

    # Display holdings
    display(HTML(f"<h2>Portfolio Status as of {date}</h2>"))
    display(HTML(df.to_html(index=False, classes='dataframe')))

    # Display summary
    print("\nPortfolio Summary:")
    print_summary_item("Cash", summary['cash'])
    print_summary_item("Total Value", summary['total_value'])
    print_summary_item("Total Market Value", summary['total_market_value'])
    print_summary_item("Total Deposits", summary['total_deposits'])

    # Display Unrealized Gain/Loss with percentage
    total_unrealized_pnl = summary['total_unrealized_pnl']
    total_unrealized_return = summary['total_unrealized_return']
    print(f"Unrealized Profit/Loss: ${float(total_unrealized_pnl)
          :,.2f} ({float(total_unrealized_return)*100:.2f}%)")

    total_realized_pnl = summary['total_realized_pnl']
    total_realized_return = summary['total_realized_return']
    print(f"Realized Profit/Loss: ${float(total_realized_pnl)
          :,.2f} ({float(total_realized_return)*100:.2f}%)")

    # Display daily gain
    daily_gain_loss = summary['total_daily_change']
    daily_return = summary['total_daily_return']
    print(f"Daily Gain: ${float(daily_gain_loss)
          :,.2f} ({float(daily_return)*100:.2f}%)")

    # Display closed positions
    print("\nClosed Positions:")
    display(HTML(closed_positions_df.to_html(index=False, classes='dataframe')))

    # Display partially sold positions
    print("\nPartially Sold Positions:")
    display(HTML(partially_sold_positions_df.to_html(
        index=False, classes='dataframe')))

def print_summary_item(label, value, is_percentage=False):
    if is_percentage:
        print(f"{label}: {float(value)*100:.2f}%")
    else:
        print(f"{label}: ${float(value):,.2f}")


# Example usage:

# processed_data = process_portfolio_data(portfolio_data, date="2024-03-21")
processed_data = process_portfolio_data(portfolio_data)

display_portfolio_analysis(processed_data)

Symbol,Quantity,Avg Cost,Market Price,Total Cost,Market Value,Unrealized P/L,Daily Change
AAPL,3.00,175.16,196.20,526.29,589.48,$63.19 (12.01%),$3.14 (0.54%)
DIS,20.00,84.05,93.50,1680.97,1869.97,$189.01 (11.24%),$21.30 (1.15%)
GOOGL,5.00,126.71,136.49,633.56,682.47,$48.91 (7.72%),$4.25 (0.63%)
PLTR,20.00,18.04,17.95,360.70,359.00,$-1.70 (-0.47%),$2.20 (0.62%)
QQQ,2.00,363.03,407.76,726.46,815.96,$89.50 (12.32%),$4.15 (0.51%)
SPY,3.01,439.46,471.85,1320.72,1418.07,$97.35 (7.37%),$8.57 (0.61%)
TSLA,30.00,215.75,257.22,6472.38,7716.60,"$1,244.22 (19.22%)",$154.20 (2.04%)
TSM,10.02,88.70,103.12,888.38,1032.88,$144.50 (16.27%),$9.35 (0.91%)



Portfolio Summary:
Cash: $592.13
Total Value: $15,076.56
Total Market Value: $14,484.43
Total Deposits: $13,000.00
Unrealized Profit/Loss: $1,874.97 (14.87%)
Realized Profit/Loss: $194.61 (1.50%)
Daily Gain: $207.16 (1.39%)

Closed Positions:


Symbol,Close Date,Quantity,Cost Basis,Realized P/L,Realized Return
SONY,2023-10-24,2.0,165.8,$1.93,1.16%
PLTR,2023-11-03,30.0,440.0,$109.17,24.81%
AMD,2023-11-14,3.0,298.02,$63.14,21.19%



Partially Sold Positions:


Symbol,Last Sell Date,Quantity,Cost Basis,Realized P/L,Realized Return
DIS,2023-11-09,3.0,252.1447826,$20.37,8.08%


In [6]:
def plot_stock_proportions(portfolio_data, date=None, show_values=True, main_threshold=3.0):
    date = NOTEBOOK_DATE or date or max(portfolio_data.keys())
    if date not in portfolio_data:
        raise ValueError(f"No data available for date: {date}")

    holdings = portfolio_data[date]['holdings']
    labels = list(holdings.keys())
    values = [float(data['market_value']) for data in holdings.values()]

    total = sum(values)
    percentages = [(value / total) * 100 for value in values]

    main_holdings = list(zip(labels, values, percentages))
    main_holdings.sort(key=lambda x: x[2], reverse=True)

    other_holdings = [h for h in main_holdings if h[2] < main_threshold]
    if len(other_holdings) > 1:
        others_value = sum(v for _, v, _ in other_holdings)
        others_percentage = sum(p for _, _, p in other_holdings)
        other_symbols_info = "<br>".join(f"<span style='font-family: monospace;'>{l:<5}\t{
                                         p:>6.2f}%</span>" for l, _, p in other_holdings)
        main_holdings = [h for h in main_holdings if h[2] >=
                         main_threshold] + [("Others", others_value, others_percentage)]

    main_labels, main_values, main_percentages = zip(*main_holdings)

    # Create custom hover text
    hover_text = [f"{label}<br>${value:,.2f}<br>{percentage:.2f}%" for label,
                  value, percentage in zip(main_labels, main_values, main_percentages)]
    if "Others" in main_labels:
        others_index = main_labels.index("Others")
        hover_text[others_index] += f"<br><br>{other_symbols_info}"

    fig = go.Figure(data=[go.Pie(labels=main_labels,
                                 values=main_percentages,
                                 textinfo='label+percent',
                                 hovertext=hover_text,
                                 hoverinfo='text')])

    fig.update_layout(
        title=f'Stock Proportions by Value on {date}',
        showlegend=True,
        legend=dict(title="Holdings")
    )

    if show_values:
        fig.update_traces(
            text=[f'${value:,.2f}' for value in main_values], textposition='inside')

    return fig


# Example usage:
# plot_stock_proportions(portfolio_data, date="2024-04-10", show_values=False)
plot_stock_proportions(portfolio_data)

In [7]:
def plot_sector_proportions(portfolio_data, sectors, date=None, max_label_length=15):
    date = NOTEBOOK_DATE or date or max(portfolio_data.keys())
    if date not in portfolio_data:
        raise ValueError(f"No data available for date: {date}")

    holdings = portfolio_data[date]['holdings']
    sector_values = defaultdict(float)
    total_value = 0

    for symbol, data in holdings.items():
        sector = sectors.get(symbol, 'ETF')
        market_value = float(data['market_value'])
        sector_values[sector] += market_value
        total_value += market_value

    labels = list(sector_values.keys())
    sizes = list(sector_values.values())
    percentages = [value / total_value * 100 for value in sizes]

    # Wrap long labels
    wrapped_labels = [wrap_label(label, max_label_length) for label in labels]

    hover_text = [f"{sector}<br>${value:,.2f}<br>{percentage:.2f}%"
                  for sector, value, percentage in zip(labels, sizes, percentages)]

    fig = go.Figure(data=[go.Pie(
        labels=wrapped_labels,
        values=sizes,
        text=[f'${value:,.2f}' for value in sizes],
        hovertext=hover_text,
        hoverinfo='text',
        textinfo='label+percent'
    )])

    fig.update_layout(
        title=f'Sector Proportions on {date}',
        showlegend=True,
        legend=dict(title="Sectors")
    )

    return fig


def wrap_label(label, max_length):
    if len(label) <= max_length:
        return label

    # Find the last space before max_length
    split_index = label.rfind(' ', 0, max_length)
    if split_index == -1:  # If no space found, split at max_length
        split_index = max_length

    return f"{label[:split_index]}<br>{label[split_index:].strip()}"

# Example usage:
# plot_sector_proportions(portfolio_data, sectors, date="2024-04-10")
plot_sector_proportions(portfolio_data, sectors)

In [8]:
def plot_portfolio_growth(portfolio_data, end_date=None):
    dates = sorted(portfolio_data.keys())

    end_date = NOTEBOOK_DATE or end_date or max(portfolio_data.keys())
    if end_date is None:
        end_date = dates[-1]
    else:
        end_date = str(end_date)
        if end_date not in dates:
            raise ValueError(f"No data available for date: {end_date}")

    start_date = pd.to_datetime(dates[0])
    dates = [d for d in dates if pd.to_datetime(
        d) >= start_date and pd.to_datetime(d) <= pd.to_datetime(end_date)]

    df_plot = pd.DataFrame({
        'Date': pd.to_datetime(dates),
        'Total Market Value': [float(portfolio_data[d]['summary']['total_market_value']) for d in dates],
        'Cash': [float(portfolio_data[d]['summary']['cash']) for d in dates],
        'Unrealized P/L': [float(portfolio_data[d]['summary']['total_unrealized_pnl']) for d in dates],
        'Realized P/L': [float(portfolio_data[d]['summary']['total_realized_pnl']) for d in dates],
        'Daily P/L': [float(portfolio_data[d]['summary']['total_daily_change']) for d in dates],
        'Unrealized Return (%)': [float(portfolio_data[d]['summary']['total_unrealized_return']) * 100 for d in dates],
        'Realized Return (%)': [float(portfolio_data[d]['summary']['total_realized_return']) * 100 for d in dates],
        'Daily Return (%)': [float(portfolio_data[d]['summary']['total_daily_return']) * 100 for d in dates],
    })

    returns = calculate_returns(df_plot, pd.to_datetime(end_date))

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces in the desired order
    add_realized_pl_trace(fig, df_plot)
    add_unrealized_pl_trace(fig, df_plot)
    add_unrealized_return_trace(fig, df_plot)
    add_total_market_value_trace(fig, df_plot)
    add_sp500_trace(fig, df_plot)

    add_performance_annotation(fig, returns)

    fig.update_layout(
        title={'text': f'Portfolio Performance as of {end_date}<br>',
               'xanchor': 'left',
               'yanchor': 'top'},
        xaxis_title='Date',
        hovermode='x unified',
        showlegend=True,
        xaxis=dict(
            rangeslider=dict(visible=False),
            rangeselector=dict(
                buttons=get_range_selector_buttons(df_plot)
            )
        ),
        margin=dict(t=120),
        barmode='stack'
    )

    fig.update_yaxes(title_text="Return (%)",
                     ticksuffix='%', secondary_y=False)
    fig.update_yaxes(title_text="Value ($)", tickprefix='$', secondary_y=True)

    return fig

def calculate_returns(df_plot, latest_date):
    returns = {}
    periods = [('1D', 1), ('1W', 7), ('1M', 30), ('3M', 90),
               ('6M', 180), ('YTD', None), ('1Y', 365), ('Max', None)]

    for period, days in periods:
        if period == '1D':
            # Use the daily return directly for 1D
            returns[period] = df_plot[df_plot['Date'] ==
                                      latest_date]['Daily Return (%)'].values[0]
        elif period == 'YTD':
            start_date = pd.Timestamp(year=latest_date.year, month=1, day=1)
        elif period == 'Max':
            start_date = df_plot['Date'].min()
        else:
            start_date = latest_date - pd.Timedelta(days=days)

        if period != '1D' and start_date >= df_plot['Date'].min():
            returns[period] = df_plot[df_plot['Date'] == latest_date]['Unrealized Return (%)'].values[0] - \
                df_plot[df_plot['Date'] >=
                        start_date]['Unrealized Return (%)'].iloc[0]

    return returns


def add_total_market_value_trace(fig, df_plot):
    fig.add_trace(
        go.Scatter(x=df_plot['Date'], y=df_plot['Total Market Value'].round(2),
                   line=dict(color='green', width=2), name='Total Market Value', visible=True,
                   hovertemplate='$%{y:,.2f}\t(Cash: $%{customdata:,.2f})',
                   customdata=df_plot['Cash'].round(2)),
        secondary_y=True
    )


def add_unrealized_return_trace(fig, df_plot):
    fig.add_trace(
        go.Scatter(x=df_plot['Date'], y=df_plot['Unrealized Return (%)'].round(2), fill='tozeroy',
                   fillcolor='rgba(255,0,0,0.2)', line=dict(color='red', width=2), name='Unr. Return',
                   hovertemplate='%{y:.2f}%\t(Daily: %{customdata:.2f}%)',
                   customdata=df_plot['Daily Return (%)'].round(2), visible=True),
        secondary_y=False
    )


def add_unrealized_pl_trace(fig, df_plot):
    fig.add_trace(
        go.Bar(x=df_plot['Date'], y=df_plot['Unrealized P/L'].round(2), name='Unr. P/L',
               marker_color='gray', opacity=0.9,
               hovertemplate='$%{y:.2f} (Daily: $%{customdata:.2f})',
               customdata=df_plot['Daily P/L'].round(2)),
        secondary_y=True
    )

def add_realized_pl_trace(fig, df_plot):
    fig.add_trace(
        go.Bar(x=df_plot['Date'], y=df_plot['Realized P/L'].round(2), name='Realized P/L',
               marker_color='black', opacity=0.9,
               hovertemplate='$%{y:.2f} (Total: $%{customdata:.2f})',
               customdata=(df_plot['Realized P/L'] + df_plot['Unrealized P/L']).round(2)),
        secondary_y=True
    )


def add_sp500_trace(fig, df_plot):
    sp500_data = fetch_sp500_data(df_plot['Date'].min(), df_plot['Date'].max())
    sp500_performance = (
        (sp500_data['Close'] - sp500_data['Close'].iloc[0]) / sp500_data['Close'].iloc[0]) * 100
    fig.add_trace(
        go.Scatter(x=sp500_data.index, y=sp500_performance.round(2), line=dict(color='blue', width=2),
                   name='S&P 500', visible=True,
                   hovertemplate='S&P 500: %{y:.2f}%<extra></extra>'),
        secondary_y=False
    )


def add_performance_annotation(fig, returns):
    performance_text = "Portfolio Return:<br>"
    for i, (period, value) in enumerate(returns.items()):
        if i % 2 == 0 and i != 0:
            performance_text += "<br>"
        color = 'green' if value >= 0 else 'red'
        sign = '+' if value > 0 else ('-' if value < 0 else ' ')
        formatted_value = f"{sign}{abs(value):6.2f}%"
        performance_text += f"<span style='display: inline-block; width: 70px;'>{
            period:>3}: <span style='color: {color};'>{formatted_value:>9}</span></span>"
        if i % 2 == 0 and i != len(returns) - 1:
            performance_text += " | "

    fig.add_annotation(
        x=1, y=1.05, xref='paper', yref='paper', text=performance_text, showarrow=False,
        font=dict(size=10, family="monospace"), align='left', xanchor='right', yanchor='bottom',
        bgcolor='rgba(255,255,255,0.8)', bordercolor='rgba(0,0,0,0.5)', borderwidth=1, borderpad=4
    )

def get_range_selector_buttons(df_plot):
    start_date = df_plot['Date'].min()
    end_date = df_plot['Date'].max()
    date_range = end_date - start_date

    buttons = []

    if date_range >= timedelta(days=7):
        buttons.append(
            dict(count=7, label="1W", step="day", stepmode="backward"))

    if date_range >= timedelta(days=30):
        buttons.append(
            dict(count=1, label="1M", step="month", stepmode="backward"))

    if date_range >= timedelta(days=90):
        buttons.append(
            dict(count=3, label="3M", step="month", stepmode="backward"))

    if date_range >= timedelta(days=180):
        buttons.append(
            dict(count=6, label="6M", step="month", stepmode="backward"))

    if end_date.year > start_date.year:
        buttons.append(dict(count=1, label="YTD",
                       step="year", stepmode="todate"))

    if date_range >= timedelta(days=365):
        buttons.append(
            dict(count=1, label="1Y", step="year", stepmode="backward"))

    buttons.append(dict(step="all", label="MAX"))

    return buttons


def fetch_sp500_data(start_date, end_date):
    end_date = pd.to_datetime(end_date) + timedelta(days=1)
    sp500 = yf.Ticker("^GSPC")
    sp500_data = sp500.history(start=start_date, end=end_date)
    return sp500_data[['Close']]

# Example usage:
# To plot for the latest date:
# plot_portfolio_growth(portfolio_data)

# To plot for a specific date:
# plot_portfolio_growth(portfolio_data, end_date="2024-03-15")

In [9]:
plot_portfolio_growth(portfolio_data)

In [10]:
def plot_asset_allocation_over_time(portfolio_data, sort_by='percentage', end_date=None):
    dates = sorted(portfolio_data.keys())

    end_date = NOTEBOOK_DATE or end_date or max(portfolio_data.keys())
    if end_date is None:
        end_date = dates[-1]
    else:
        end_date = str(end_date)
        if end_date not in dates:
            raise ValueError(f"No data available for date: {end_date}")

    # Filter dates to include only up to the end_date
    dates = [date for date in dates if date <= end_date]

    latest_date = end_date
    active_stocks = get_active_stocks(portfolio_data[latest_date]['holdings'])
    data = calculate_asset_proportions(portfolio_data, dates, active_stocks)

    fig = go.Figure()
    add_asset_traces(fig, dates, data, portfolio_data, sort_by)
    update_layout(fig, sort_by, latest_date)

    # Set x-axis range
    fig.update_xaxes(range=[dates[0], latest_date])

    return fig


def get_active_stocks(latest_holdings):
    return [stock for stock, info in latest_holdings.items()
            if float(info['market_value']) > 0]


def calculate_asset_proportions(portfolio_data, dates, active_stocks):
    data = {stock: [] for stock in active_stocks}
    data['Cash'] = []

    for date in dates:
        holdings = portfolio_data[date]['holdings']
        valid_holdings = {stock: info for stock, info in holdings.items()
                          if not math.isnan(float(info['market_value']))}

        cash = float(portfolio_data[date]['summary']['cash'])
        total_value = sum(float(info['market_value'])
                          for info in valid_holdings.values()) + cash

        for stock in active_stocks:
            if stock in valid_holdings:
                value = float(valid_holdings[stock]['market_value'])
                proportion = (value / total_value) * \
                    100 if total_value > 0 else 0
            else:
                proportion = 0
            data[stock].append(proportion)

        cash_proportion = (cash / total_value) * 100 if total_value > 0 else 0
        data['Cash'].append(cash_proportion)

    return data


def add_asset_traces(fig, dates, data, portfolio_data, sort_by):
    if sort_by == 'date':
        sorted_stocks = sort_by_add_date(portfolio_data, data.keys())
    else:  # sort_by == 'percentage'
        sorted_stocks = sort_by_percentage(data)

    # Remove Cash from sorted_stocks if it's there
    sorted_stocks = [stock for stock in sorted_stocks if stock != 'Cash']

    # Add traces in reverse order, with Cash always at the end (top of the stack)
    for stock in reversed(sorted_stocks):
        add_trace(fig, dates, data, stock)

    # Always add Cash last (it will appear at the top)
    add_trace(fig, dates, data, 'Cash', '#FFD700')


def sort_by_add_date(portfolio_data, stocks):
    first_appearance = {}
    for date in sorted(portfolio_data.keys()):
        for stock in stocks:
            if stock not in first_appearance and stock != 'Cash':
                if stock in portfolio_data[date]['holdings']:
                    first_appearance[stock] = date
    return sorted(first_appearance, key=first_appearance.get)


def sort_by_percentage(data):
    latest_percentages = {stock: values[-1]
                          for stock, values in data.items() if stock != 'Cash'}
    return sorted(latest_percentages, key=latest_percentages.get, reverse=True)


def add_trace(fig, dates, data, stock, color=None):
    proportions = data[stock]
    hover_text = [f"{stock}: {proportion:.2f}%" if not math.isnan(proportion)
                  else f"{stock}: N/A" for proportion in proportions]

    fig.add_trace(go.Scatter(
        x=dates,
        y=proportions,
        mode='lines',
        stackgroup='one',
        name=stock,
        hoverinfo='text',
        text=hover_text,
        line=dict(color=color) if color else dict(),
    ))


def update_layout(fig, sort_by, latest_date):
    title = 'Asset Allocation Over Time (Sorted by {}) as of {}'.format(
        'Latest Percentage' if sort_by == 'percentage' else 'Add Date',
        latest_date
    )
    fig.update_layout(
        title=title,
        xaxis_title='Date',
        yaxis=dict(
            title='Proportion',
            tickformat='.0%',
            range=[0, 100],
            tickvals=[0, 20, 40, 60, 80, 100],
            ticktext=['0%', '20%', '40%', '60%', '80%', '100%']
        ),
        legend_title='Assets',
        hovermode='x unified'
    )


# Usage examples:
# Default (latest date)
plot_asset_allocation_over_time(portfolio_data)

# Specific date
# fig = plot_asset_allocation_over_time(portfolio_data, end_date="2024-03-15")
# fig.show()

# Sort by add date
# fig = plot_asset_allocation_over_time(portfolio_data, sort_by='date')
# fig.show()

# Specific date and sort by add date
# fig = plot_asset_allocation_over_time(portfolio_data, sort_by='date', end_date="2024-03-15")
# fig.show()

In [11]:
def generate_html_report(report_date=None):
    dates = sorted(portfolio_data.keys())
    
    report_date = NOTEBOOK_DATE or report_date or max(portfolio_data.keys())
    if report_date is None:
        report_date = dates[-1]  # Use the latest date by default
    else:
        report_date = str(report_date)  # Ensure the date is a string
        if report_date not in dates:
            raise ValueError(f"No data available for date: {report_date}")

    stock_proportions_plot = plot_stock_proportions(
        portfolio_data, report_date)
    sector_proportions_plot = plot_sector_proportions(
        portfolio_data, sectors, report_date)
    portfolio_growth_plot = plot_portfolio_growth(
        portfolio_data, end_date=report_date)
    asset_allocation_over_time_plot = plot_asset_allocation_over_time(
        portfolio_data, end_date=report_date)

    for plot in [stock_proportions_plot, sector_proportions_plot, portfolio_growth_plot, asset_allocation_over_time_plot]:
        plot.update_layout(font=dict(family="monospace", weight='bold'))

    report_data = portfolio_data[report_date]
    summary = report_data['summary']
    holdings = report_data['holdings']
    closed_positions = report_data['closed_positions']
    partially_sold_positions = report_data['partially_sold_positions']

    # Function to safely convert string to float
    def safe_float(value):
        try:
            return float(value)
        except (ValueError, TypeError):
            return 0.0

    # Function to format currency values
    def format_currency(value):
        return f"${safe_float(value):,.2f}"

    # Function to format percentage values
    def format_percentage(value):
        return f"{safe_float(value) * 100:.2f}%"

    # Function to combine PNL and return
    def combine_pnl_return(pnl, ret):
        return f"{format_currency(pnl)} ({format_percentage(ret)})"

    # Process holdings
    holdings_df = pd.DataFrame(holdings).T.reset_index()
    holdings_df.columns = ['Symbol', 'Quantity', 'Cost Basis', 'Avg Cost', 'Market Price',
                           'Market Value', 'Unrealized P/L', 'Unrealized Return', 'Daily Change', 'Daily Return']

    # Apply formatting to numeric columns
    for col in ['Quantity', 'Cost Basis', 'Avg Cost', 'Market Price', 'Market Value']:
        holdings_df[col] = holdings_df[col].apply(lambda x: f"{safe_float(x):.2f}")

    holdings_df['Unrealized P/L'] = holdings_df.apply(
        lambda row: combine_pnl_return(row['Unrealized P/L'], row['Unrealized Return']), axis=1)
    holdings_df['Daily Change'] = holdings_df.apply(
        lambda row: combine_pnl_return(row['Daily Change'], row['Daily Return']), axis=1)
    holdings_df = holdings_df.drop(columns=['Unrealized Return', 'Daily Return'])

    # Process closed positions
    closed_positions_df = pd.DataFrame(closed_positions).T.reset_index()
    closed_positions_df.columns = [
        'Symbol', 'Close Date', 'Quantity', 'Cost Basis', 'Realized P/L', 'Realized Return']

    # Apply formatting to numeric columns
    for col in ['Quantity', 'Cost Basis']:
        closed_positions_df[col] = closed_positions_df[col].apply(lambda x: f"{safe_float(x):.2f}")

    closed_positions_df['Realized P/L'] = closed_positions_df.apply(
        lambda row: combine_pnl_return(row['Realized P/L'], row['Realized Return']), axis=1)
    closed_positions_df = closed_positions_df.drop(columns=['Realized Return'])

    # Process partially sold positions
    partially_sold_positions_df = pd.DataFrame(partially_sold_positions).T.reset_index()
    partially_sold_positions_df.columns = [
        'Symbol', 'Last Sell Date', 'Quantity', 'Cost Basis', 'Realized P/L', 'Realized Return']

    # Apply formatting to numeric columns
    for col in ['Quantity', 'Cost Basis']:
        partially_sold_positions_df[col] = partially_sold_positions_df[col].apply(lambda x: f"{safe_float(x):.2f}")

    partially_sold_positions_df['Realized P/L'] = partially_sold_positions_df.apply(
        lambda row: combine_pnl_return(row['Realized P/L'], row['Realized Return']), axis=1)
    partially_sold_positions_df = partially_sold_positions_df.drop(columns=['Realized Return'])

    # Format date columns
    date_columns = ['Close Date', 'Last Sell Date']
    for df in [closed_positions_df, partially_sold_positions_df]:
        for col in date_columns:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col]).dt.strftime('%Y-%m-%d')

    portfolio_table = holdings_df.to_html(index=False, classes='dataframe')
    closed_positions_table = closed_positions_df.to_html(index=False, classes='dataframe')
    partially_sold_positions_table = partially_sold_positions_df.to_html(index=False, classes='dataframe')

    format_dict = {
        'date': report_date,
        'portfolio_table': portfolio_table,
        'portfolio_growth_plot': to_html(portfolio_growth_plot, full_html=False, include_plotlyjs=False, config={'responsive': True}),
        'stock_proportions_plot': to_html(stock_proportions_plot, full_html=False, include_plotlyjs=False, config={'responsive': True}),
        'sector_proportions_plot': to_html(sector_proportions_plot, full_html=False, include_plotlyjs=False, config={'responsive': True}),
        'asset_allocation_over_time_plot': to_html(asset_allocation_over_time_plot, full_html=False, include_plotlyjs=False, config={'responsive': True}),
        'date_options': ' '.join(f'<option value="{date}">{date}</option>' for date in dates),
        'cash': format_currency(summary['cash']),
        'total_value': format_currency(summary['total_value']),
        'total_market_value': format_currency(summary['total_market_value']),
        'unrealized_pnl': combine_pnl_return(summary['total_unrealized_pnl'], summary['total_unrealized_return']),
        'daily_change': combine_pnl_return(summary['total_daily_change'], summary['total_daily_return']),
        'total_realized_pnl': combine_pnl_return(summary['total_realized_pnl'], summary['total_realized_return']),
        'total_deposits': format_currency(summary['total_deposits']),
        'closed_positions_table': closed_positions_table,
        'partially_sold_positions_table': partially_sold_positions_table,
    }

    template_path = 'reports/templates/portfolio_report_template.html'
    with open(template_path, 'r') as template_file:
        html_template = template_file.read()

    css_path = 'reports/templates/portfolio_report_styles.css'
    with open(css_path, 'r') as css_file:
        css_content = css_file.read()

    css_content_escaped = css_content.replace('{', '{{').replace('}', '}}')
    html_template = html_template.replace('/* CSS_PLACEHOLDER */', css_content_escaped)
    html_template = re.sub(r'\{\s*(\w+)\s*\}', r'{\1}', html_template)
    html_content = html_template.format(**format_dict)

    os.makedirs('reports', exist_ok=True)
    
    if report_date == max(portfolio_data.keys()):
        filename = f'portfolio_report_latest.html'
    else:
        filename = f'portfolio_report_{report_date}.html'
    with open(os.path.join('reports', filename), 'w') as f:
        f.write(html_content)

    print(f"HTML report generated: reports/{filename}")

# Usage examples:
# Generate report for the latest date
generate_html_report()

# Generate report for a specific date
# generate_html_report("2024-03-15")

HTML report generated: reports/portfolio_report_2023-12-19.html
